# Background:
- When looking at the data.bin created using 2-step nonrigid registration, with smoothing, it was really not registered.
- Nonrigid registration also made session-to-session registration bad.
# Purpose:
- Try single step, rigid registration.
    - Run some examples up to the signal calculation. See how long it takes, and look at the data quality.
    - If it is feasible, re-process every data
        - Only the necessary data first.
- This will be a good chance to organize how things are processed.
    - Rigid registration
    - Add 'max_proj'
    - Detect ROIs using cellpose
        - Merge across different types of projection images
    - Extract signals
- Save new results in 'plane1' folder

In [2]:
from suite2p import run_s2p, default_ops
from suite2p.io.binary import BinaryFile
from suite2p.detection.denoise import pca_denoise
from suite2p.detection import utils

from scripts import cellpose_roi_detection, roi_collation, roi_remove_border, save_signal

import time
import numpy as np
import os
from pathlib import Path
import glob
base_dir = Path('E:/TPM/JK/h5')

In [4]:
def add_max_proj(ops_fn):
    ops = np.load(ops_fn, allow_pickle=True).item()
    if 'max_proj' in ops.keys():
        print("'max_proj' already in ops.")
    else:
        print("Add 'max_proj' to the ops:")

        bin_size = int(max(1, ops['nframes'] // ops['nbinned'], np.round(ops['tau'] * ops['fs'])))
        if not os.path.isfile(ops['reg_file']):
            reg_file = str(Path(ops['save_path']) / 'data.bin')
            if os.path.isfile(reg_file):
                ops['reg_file'] = reg_file
            else:
                raise FileNotFoundError(f'{ops["reg_file"]} or {reg_file} not found')
        
        with BinaryFile(read_filename=ops['reg_file'], Ly=ops['Ly'], Lx=ops['Lx']) as f:
            t0 = time.time()
            mov = f.bin_movie(
                bin_size=bin_size,
                bad_frames=ops.get('badframes'),
                y_range=ops['yrange'],
                x_range=ops['xrange'],
            )
            print('Binned movie [%d,%d,%d] in %0.2f sec.' % (mov.shape[0], mov.shape[1], mov.shape[2], time.time() - t0))
            
        if ops.get('denoise', 1):
            print('PCA denosing...')
            mov = pca_denoise(mov, block_size=[ops['block_size'][0]//2, ops['block_size'][1]//2],
                                n_comps_frac = 0.5)
        else:
            print('No denoising')
        t0 = time.time()
        mov = utils.temporal_high_pass_filter(mov=mov, width=int(ops['high_pass']))
        max_proj = mov.max(axis=0)
        ops['max_proj'] = max_proj
        processing_time = time.time() - t0
        print(f'Added max_proj in {processing_time:.2f} sec.')

        # save ops
        np.save(ops_fn, ops)

In [3]:
ops = default_ops()
ops['nimg_init']

300

In [5]:
from importlib import reload
reload(save_signal)
mice = [25,  27,  30,  36,  37,  38,  39,  41,  52,  53,  54,  56]
zoom = [2,   2,   2,   1.7, 1.7, 1.7, 1.7, 1.7, 1.7, 1.7, 1.7, 1.7]
freq = [7.7, 7.7, 7.7, 7.7, 6.1, 6.1, 6.1, 6.1, 7.7, 7.7, 7.7, 1.7 ]
planes= range(1,9)
ops = default_ops()
ops['tau'] = 1.5
ops['look_one_level_down'] = False
ops['do_bidiphase'] = True
# ops['nimg_init'] = 1000
ops['batch_size'] = 3000
ops['save_mat'] = True
ops['save_NWB'] = False # for now. Need to set up parameters and confirm it works
# Do not run ROI detection - will run it separately using cellpose
ops['roidetect'] = False

mi = 6
sessions = [11]
mouse = mice[mi]
ops['fs'] = freq[mi]
ops['zoom'] = zoom[mi]
ops['umPerPix'] = 1.4/ops['zoom']
for plane in range(1,2):
    mouseDir = base_dir / f'{mouse:03}'
    planeDir = mouseDir / f'plane_{plane}/'
    for session_num in sessions:
        tempFnList = glob.glob(str(planeDir / f'{mouse:03}_{session_num:03}_*_plane_{plane}.h5'))
        repFn = tempFnList[0]
        db = {'h5py': repFn,
                'h5py_key': ['data'],
                'data_path': [],
                'save_path0': str(planeDir / 'test'),
                'save_folder': f'{session_num:03}',
                'fast_disk': 'C:/suite2p_fast_disk/',
                'move_bin': True,
                'do_registration': 1, 
                # Addendum for low SNR
                # 'two_step_registration': True,
                # 'keep_movie_raw': True,
                # 'smooth_sigma_time': 2
            }
        # run_s2p(ops,db)  # CANNOT be parallelized
        # ops_fn = db['save_path0'] + '/' + db['save_folder'] + '/plane0/ops.npy'
        
        # add_max_proj(ops_fn)
        
        # ROI detection from multiple projection files
        # cellpose_roi_detection.generate_multi_cp_masks(base_dir, mouse, plane, session_num, test_dir=True)
        # roi_collation.process_mask_merging(base_dir, mouse, plane, session_num, test_dir=True)
        # roi_remove_border.border_filter_mask(base_dir, mouse, plane, session_num, test_dir=True)
        save_signal.save_signal(base_dir, mouse, plane, session_num, test_dir=True)
        
        

{'h5py': 'E:\\TPM\\JK\\h5\\039\\plane_1\\039_011_000_plane_1.h5', 'h5py_key': ['data'], 'data_path': [], 'save_path0': 'E:\\TPM\\JK\\h5\\039\\plane_1\\test', 'save_folder': '011', 'fast_disk': 'C:/suite2p_fast_disk/', 'move_bin': True, 'do_registration': 2}
h5
time 41.84 sec. Wrote 5835 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 5835 frames
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 23.84 sec.
added enhanced mean image
----------- Total 786.15 sec
Registration metrics, 46.90 sec.
moving binary files to save_path
Plane 0 processed in 839.69 sec (can open in GUI).
total = 888.67 sec.
TOTAL RUNTIME 888.68 sec
Add 'max_proj' to the ops:
Binned movie [605,427,583] in 12.82 sec.
No denoising
Added max_proj in 0.64 sec.
Processing 039_plane_1_011...
039_plane_1_011 done: 12.80 min

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numba\core\ir_utils.py:2139: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'cell_ipix' of function 'matmul_traces'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\..\..\Anaconda3\envs\suite2p\lib\site-packages\suite2p\extraction\extract.py", line 105:
@njit(parallel=True)
def matmul_traces(Fi, data, cell_ipix, cell_lam):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numba\core\ir_utils.py:2139: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'cell_lam' of function 'matmul_traces'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference

Extracted fluorescence from 106 ROIs in 5835 frames, 18.12 sec.


c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


save_signal took 27.41 seconds


## It will take about 2 weeks to run all the sessions and planes
- Registration for 24000 frames (more than double compared to most of the sessions) 
    - Took 10 min
    - 5835 frames from a large FOV took 15 min.
    - The rest will take about 10-15 min but can be parallelized.
        - Adding max projection took 13 min for a large FOV (5835 frames)
        - The rest takes about 1 min.
- Previous paper was processed the same way and the results were clear.
- Proceed for now.
- At least check if it will remove weird signal pattern

In [8]:
session_num

11

In [9]:
mouse=39
session_num=18
for plane in range(1,9):
    save_signal.save_signal(base_dir, mouse, plane, session_num)

Masks made in 1.87 sec.
Extracted fluorescence from 148 ROIs in 3618 frames, 10.87 sec.


c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\TPM\\JK\\h5\\039\\plane_1\\039_018_plane_1_frame_time.pkl'